In [1]:
import numpy as np
import pandas as pd
from scipy.stats import uniform, norm
from bayesian_estimation_noise_testing import BayesianEstimation, LogLikelihoods
from utils import *


# Harricana River

## Load systematic data

In [2]:
annual_maxima_csv = r"C:\ISYE6420\Homework\Project\data\Harricana_River_at_Amos.csv"

df = pd.read_csv(annual_maxima_csv)
df["zstd"] = -norm.ppf(df["Plotting_Position"])

# convert flow to array
data = df["Flow"].to_xarray()

## Bayesian estimation

In [3]:
# Compute the prior limits
mu_min, mu_max = 0, 4
sigma_min, sigma_max = 0, 2
gamma_min, gamma_max = -2, 2

# Define the prior distributions
priors = [
    lambda: uniform.rvs(loc=mu_min, scale=mu_max - mu_min),
    lambda: uniform.rvs(loc=sigma_min, scale=sigma_max - sigma_min),
    lambda: uniform.rvs(loc=gamma_min, scale=gamma_max - gamma_min)
]

prior_limits = [
    (mu_min, mu_max),
    (sigma_min, sigma_max),
    (gamma_min, gamma_max)
]

log_likelihood_func = LogLikelihoods(data).lp3
bayesian_estimation_lp3 = BayesianEstimation(
    data=data, 
    log_likelihood_func=log_likelihood_func, 
    prior=priors, 
    prior_limits=prior_limits, 
    seed=253
    )

# Run the DEMCz sampler
samples, acceptance_rates = bayesian_estimation_lp3.demcz_sampler(
    num_chains=5, 
    iterations=44000, 
    burn_in=4000, 
    jump=0.97163, 
    jump_threshold=0.1,
    noise=1e-3,
    snooker_threshold=0.1,
    thinning_interval=20
    )
np.save(r'C:\ISYE6420\Homework\Project\data\HRA\HRA_bayesian_samples_noise_testing.npy', samples)

# Display the acceptance rates
print("Acceptance rates:\n")
print(acceptance_rates)

# Display the summary statistics
summaries_lp3 = bayesian_estimation_lp3.calculate_log_likelihoods(
    variable=["mu", "sigma", "gamma"]
    )
summaries_lp3.to_csv(r'C:\ISYE6420\Homework\Project\data\HRA\HRA_bayesian_lp3_summaries_noise_testing.csv', index=False)
    
posterior_mode = summaries_lp3.loc[summaries_lp3['LogLikelihood'].idxmax()]
    
print("\nPosterior mode:\n")
print(posterior_mode)

Acceptance rates:

[0.50893182 0.51265909 0.50445455 0.51472727 0.5175    ]

Posterior mode:

mu                 2.269563
sigma              0.105944
gamma             -0.023311
LogLikelihood   -361.192712
Name: 3975, dtype: float64
